<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Custom-functions" data-toc-modified-id="Custom-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Custom functions</a></span></li><li><span><a href="#Environment-variables" data-toc-modified-id="Environment-variables-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Environment variables</a></span></li><li><span><a href="#Load-and-process-dataset" data-toc-modified-id="Load-and-process-dataset-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load and process dataset</a></span></li></ul></div>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/My Drive/Colab Notebooks/accent/src

### Import packages

In [1]:
import re, zipfile, os, io, time, string, numpy as np, matplotlib.ticker as ticker, \
            matplotlib.pyplot as plt

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# from utils import process_raw, generate_input, get_max_len, process_data, convert

from token_list import strip_tokens
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, Input, Embedding, Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.experimental import export_saved_model, load_from_saved_model

In [2]:
# x = ['Khoan no cua KH se den han vao 01/08/2019. So tien  2.999.898 VND, TK 12345678912. Thong tin chi tiet, lien he 1900636633. Cam on',
# 'Khoan no cua KH se den han trong 2 ngay toi. So tien  2.999.898 VND, TK 12345678912. Bo qua neu da TT. Thong tin chi tiet, lien he 1900636633. Cam on',
# 'Khoan no cua KH da qua han 1 ngay. TK 12345678912, so tien2.999.898 VND. Bo qua neu da TT. Thong tin chi tiet, lien he 18006288',
# 'Khoan no cua KH da qua han 6 ngay va bi tinh phat 250,000. TK 12345678912, so tien 2.999.898VND. Vui long TT. Thong tin chi tiet, lien he 18006288',
# 'KH da qua han 5 ky no. TK 12345678912, so tien 12.999.898VND. Vui long TT ngay lap tuc. Thong tin chi tiet, lien he 18006288',
# 'Khoan no cua KH da bi tinh phat do lien tuc vi pham. TK12345678912, so tien 12.999.898VND. Vui long TT. Thong tin chi tiet, lien he 18006288',
# 'Khoan no cua KH da qua han 91 ngay. Vui long TT toan bo29.999.898VND vao TK 12345678912 hom nay. Thong tin chi tiet, lien he 18006288',
# 'Chung toi vua nhan thanh toan khoan vay cua KH tu TK12345678912. TUY NHIEN, KH van con thieu 29.999.898VND. Vui long TT toan ngay lap tuc. Cam on']

### Custom functions

In [3]:
def process_raw(raw_data):
    raw_data = [seq.lower().strip() for seq in raw_data]

    # Creating a space between a word and the punctuation following it
    # Eg: "he is a boy." => "he is a boy ."
    raw_data = [re.sub(r"([?.!,¿])", r" \1 ", seq) for seq in raw_data]
    raw_data = [re.sub(r'[" "]+', " ", seq) for seq in raw_data]

    # Replacing everything with space except (characters, ".", "?", "!", ",")
    filtered_punctuations = string.punctuation
    exclude = [',', '!', '.', '?']

    for c in filtered_punctuations:
        if c in exclude:
            filtered_punctuations = filtered_punctuations.replace(c, '')

    table = str.maketrans('', '', filtered_punctuations)
    raw_data = [seq.translate(table) for seq in raw_data]
    
    # Append start and end tokens to sequences
    processed_raw = []
    for seq in raw_data:
        words = seq.split()
        words = [word.strip() for word in words]
        processed_raw.append(' '.join(words))

    return processed_raw

In [4]:
def generate_input(processed_raw):
    output = ''
    for char in processed_raw:
        if char in strip_tokens:
            output += strip_tokens[char]
        else:
            output += char          
    return output

In [5]:
def get_max_len(input_data, get_index=False): 
    longest = [len(data.split()) for data in input_data]
    if get_index:
        print(longest.index(max(longest)))
    return max(longest)

In [6]:
def tokenize_pad_data(data):
    tk = Tokenizer(char_level=False, filters='')
    tk.fit_on_texts(data)
    data = tk.texts_to_sequences(data)
    return data, tk

In [7]:
def convert(tokenizer, tokenized_data, send_back=False):
    original = []
    
    print('Tokenized Data: {}'.format(tokenized_data))
    
    for token in tokenized_data:
        if token != 0:
            if token in tokenizer.index_word:
                original.append(tokenizer.index_word[token])
            else:
                original.append('<unk>')
                    
    print('Original Data: {}'.format(original))
    
    if send_back:
        return original

In [8]:
def evaluate(sentence):
    attention_plot = np.zeros((max_process_seq, max_process_seq))

#     sentence = preprocess_sentence(sentence)

    inputs = [input_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_process_seq, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_tokenizer.word_index['<s>']], 0)

    for t in range(max_process_seq):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_tokenizer.index_word[predicted_id] + ' '

        if target_tokenizer.index_word[predicted_id] == '<e>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
      
    return result, sentence, attention_plot

In [9]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [10]:
def restore(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

### Environment variables

In [11]:
# Define data arguements
# Set random seed
np.random.seed(50)
data_file = '../data/raw/raw_train.txt'

tf.__version__

'2.0.0-rc1'

### Load and process dataset

In [12]:
# Load dataset
counter = 0
max_seq_len = 40
no_seq = 50000
raw_data = []

# Load raw data and read first 100000 sequences with 40 or less words
with open(data_file, 'r', encoding='utf-8') as f:    
    while counter != no_seq:
        line = f.readline()
        if 5 <= len(line.split()) <= max_seq_len:
            raw_data.append(line)
            counter += 1
        else:
            continue

In [13]:
# Shuffle raw data prior to processing
np.random.shuffle(raw_data)

In [14]:
# Process data
processed_raw = process_raw(raw_data)

input_data = [generate_input(seq) for seq in processed_raw]
target_data = ['<s> ' + seq + ' <e>' for seq in processed_raw]

print('Sample Input: {}'.format(input_data[0]))
print('Sample Target: {}'.format(target_data[0]))

Sample Input: sau khi tach ra khoi lien bang xo viet , kinh te litva lam vao khung hoang tram trong .
Sample Target: <s> sau khi tách ra khỏi liên bang xô viết , kinh tế litva lâm vào khủng hoảng trầm trọng . <e>


In [15]:
# Get max sequence length after processing
max_inp_len = get_max_len(input_data, get_index=False)
max_tar_len = get_max_len(target_data, get_index=False)

In [16]:
# Generate input data word tokenizer indexes
input_tk = Tokenizer(char_level=False, filters='')
input_tk.fit_on_texts(input_data)

# Generate target data word tokenizer indexes
target_tk = Tokenizer(char_level=False, filters='')
target_tk.fit_on_texts(target_data)

inp_vocab_size = len(input_tk.word_index)+1
tar_vocab_size = len(target_tk.word_index)+1

# Reverse vocab lookup
input_tk_rev = {w:idx for idx, w in input_tk.word_index.items()}
target_tk_rev = {w:idx for idx, w in target_tk.word_index.items()}

In [17]:
print('Input vocab size: {}'.format(inp_vocab_size))
print('Target vocab size: {}'.format(tar_vocab_size))
print('Max input sequence length: {}'.format(max_inp_len))
print('Max target sequence length : {}'.format(max_tar_len))

Input vocab size: 21615
Target vocab size: 25349
Max input sequence length: 50
Max target sequence length : 52


In [18]:
# Tokenize data
input_data = input_tk.texts_to_sequences(input_data)
target_data = target_tk.texts_to_sequences(target_data)

# Format decoder and target data
decoder_data = [seq[:-1] for seq in target_data]
target_data = [seq[1:] for seq in target_data]

# Pad data
input_data = pad_sequences(input_data, padding='post', maxlen=max_inp_len)
decoder_data = pad_sequences(decoder_data, padding='post', maxlen=max_tar_len)
target_data = pad_sequences(target_data, padding='post', maxlen=max_tar_len)

In [19]:
convert(input_tk, input_data[-1])
print('\n')
convert(target_tk, decoder_data[-1])
print('\n')
convert(target_tk, target_data[-1])

Tokenized Data: [ 53 304 357  11 742   1   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Original Data: ['ong', 'nghi', 'huu', 'nam', '2000', '.']


Tokenized Data: [   2   34 1414 2007   14 1280    1    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
Original Data: ['<s>', 'ông', 'nghỉ', 'hưu', 'năm', '2000', '.']


Tokenized Data: [  34 1414 2007   14 1280    1    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
Original Data: ['ông', 'nghỉ', 'hưu', 'năm', '2000', '.', '<e>']


In [20]:
# Split data into train/val
train_data_len = round((.9 * no_seq))
val_test_data_len = round((.1 * no_seq))

encoder_input_data, decoder_input_data, decoder_target_data = input_data[:train_data_len], decoder_data[:train_data_len], \
                                                                target_data[:train_data_len]

val_encoder_input_data, val_decoder_input_data, val_decoder_target_data = input_data[-val_test_data_len:], \
                                                        decoder_data[-val_test_data_len:], target_data[-val_test_data_len:]

print('Number of training sequences: {}'.format(len(encoder_input_data)))
print('Number of val sequences: {}'.format(len(val_encoder_input_data)))

Number of training sequences: 45000
Number of val sequences: 5000


In [21]:
encoder_input_data.shape, val_encoder_input_data.shape

((45000, 50), (5000, 50))

In [22]:
model_ckpt = ModelCheckpoint('./models/sparse_best.h5', save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=5, verbose=1)

In [23]:
epochs = 100
batch_size = 64
units = 1024
embedding_dim = 256
dropout = 0.2

In [24]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(inp_vocab_size, embedding_dim, mask_zero = True)(encoder_inputs)
# encoder_lstm = Bidirectional(LSTM(units, return_state=True))
encoder_lstm = LSTM(units, return_state=True, dropout=dropout)
# encoder_outputs, forward_hidden, forward_cell, backward_hidden, backward_cell = encoder_lstm(enc_emb)
# encoder_states = [forward_hidden, forward_cell, backward_hidden, backward_cell]
encoder_outputs, forward_hidden, forward_cell = encoder_lstm(enc_emb)
encoder_states = [forward_hidden, forward_cell]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, embedding_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# decoder_lstm = Bidirectional(LSTM(units, return_sequences=True, return_state=True))
decoder_lstm = LSTM(units, return_sequences=True, return_state=True, dropout=dropout)
# decoder_outputs, _, _, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    5533440     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    6489344     input_2[0][0]                    
______________________________________________________________________________________________

In [27]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[model_ckpt, early_stop])

Train on 36000 samples, validate on 9000 samples
Epoch 1/100
35968/36000 [============================>.] - ETA: 0s - loss: 2.9259 - acc: 0.1307
Epoch 00001: val_loss improved from inf to 2.66182, saving model to ./models/sparse_best.h5
36000/36000 [==============================] - 239s 7ms/sample - loss: 2.9256 - acc: 0.1307 - val_loss: 2.6618 - val_acc: 0.1743
Epoch 2/100
35968/36000 [============================>.] - ETA: 0s - loss: 2.4951 - acc: 0.2057
Epoch 00002: val_loss improved from 2.66182 to 2.41070, saving model to ./models/sparse_best.h5
36000/36000 [==============================] - 224s 6ms/sample - loss: 2.4951 - acc: 0.2058 - val_loss: 2.4107 - val_acc: 0.2284
Epoch 3/100
35968/36000 [============================>.] - ETA: 0s - loss: 2.2607 - acc: 0.2466
Epoch 00003: val_loss improved from 2.41070 to 2.26023, saving model to ./models/sparse_best.h5
36000/36000 [==============================] - 222s 6ms/sample - loss: 2.2607 - acc: 0.2466 - val_loss: 2.2602 - val_acc:

In [28]:
model.evaluate([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data)

5000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.3194535332679749, 0.56182253]

In [29]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(units,))
decoder_state_input_c = Input(shape=(units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# # Encode the input sequence to get the "thought vectors"
# encoder_model = Model(encoder_inputs, encoder_states)

# # Decoder setup
# # Below tensors will hold the states of the previous time step
# decoder_fo_input_h = Input(shape=(units,))
# decoder_fo_input_c = Input(shape=(units,))
# decoder_ba_input_h = Input(shape=(units,))
# decoder_ba_input_c = Input(shape=(units,))
# decoder_states_inputs = [decoder_fo_input_h, decoder_fo_input_c, decoder_ba_input_h, decoder_ba_input_c]

# dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# # To predict the next word in the sequence, set the initial states to the states from the previous time step
# decoder_outputs2, fo_h2, fo_c2, ba_h2, ba_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
# decoder_states2 = [fo_h2, fo_c2, ba_h2, ba_c2]
# decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# # Final decoder model
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs2] + decoder_states2)

In [31]:
model = load_model('./models/sparse_best.h5')

In [46]:
model.layers

In [32]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_tk.word_index['<s>']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
#         output_tokens, f_h, f_c, b_h, b_c = decoder_model.predict([target_seq] + states_value)
        output_tokens, f_h, f_c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_tk_rev[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<e>' or len(decoded_sentence) > max_tar_len):
#         if len(decoded_sentence) > max_tar_len:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
#         states_value = [f_h, f_c, b_h, b_c]
        states_value = [f_h, f_c]

    return decoded_sentence

In [63]:
rand = np.random.randint(0, len(val_encoder_input_data))
print('Input:')
print(' '.join([target_tk_rev[w] for w in val_decoder_target_data[rand] if w != 0]))
print('\n')

print('Prediction:')
decode_sequence(np.array([val_encoder_input_data[rand]]))

Input:
với vóc dáng khổng lồ và sức mạnh của mình , bò tót hầu như không có kẻ thù trong tự nhiên , ngoại trừ hổ . <e>


Prediction:


' với vóc dáng khổng lồ và sức khỏe của hồng y , thái độ'